In [1]:
# DIRECTORY SET
import os
import sys
import traceback
from pathlib import Path
base_dir=Path(os.getcwd()).parent.parent
os.chdir(base_dir)
print(os.getcwd())

# ENVIRONMENT VARIABLES
import dotenv
dotenv.load_dotenv()

# DJANGO SETUP
import django
sys.path.append(os.path.abspath(''))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "serverproject.settings")
django.setup()

# Import async modules
import asyncio
from asgiref.sync import sync_to_async

# Import display modules
from IPython.display import display, Markdown

# Import other modules
import faiss

# import reloading
from importlib import reload

# Enable autoreload
%load_ext autoreload
%autoreload 2

# Import custom modules
from destinyapp.models import StreamRecapData

from destinyapp.customlibrary import services
from destinyapp.customlibrary import utils
from destinyapp.customlibrary import controller

d:\DestinyFolder\DestinyRecaps\DestinyRecapsApi\serverproject


In [19]:
await services.web_checker.web_get_recap_image()

Driver closed


# Define id

In [ ]:
# Input parameter
video_id="OqVH_MTBQ6k"

# Simple controller function run

In [ ]:
await controller.StreamPlotController.run(video_id)

# Run the contents of the controller

In [ ]:
stream_recap_data=await utils.get_recap_data(video_id)
testing_bool=True
if not testing_bool:
    annotated_results, major_topics, minor_topics = await services.stream_plot.generate_data(stream_recap_data)
else:
    import json
    with open(f"testing_visualization_data_gen.json", "r") as f:
        data_gen=json.load(f)

    annotated_results=data_gen["annotated_results"]
    major_topics=data_gen["major_topics"]
    minor_topics=data_gen["minor_topics"]
    stream_recap_data=await utils.get_recap_data(video_id)


plot_object, annotated_results, plot_segments, category_locations = await services.stream_plot.process_data(stream_recap_data,  annotated_results, major_topics, minor_topics, video_id)

await services.stream_plot.generate_plot(plot_object)

In [ ]:
plot_object.abstractions

In [ ]:
from dataclasses import dataclass, asdict, is_dataclass
from typing import Any

In [ ]:
from dataclasses import dataclass, asdict, is_dataclass
from typing import Any
import json
plot_object_json=json.dumps(asdict(plot_object))
with open(f"plot_object_{video_id}.json", "w") as f:
    f.write(plot_object_json)


In [ ]:
stream_recap_data=await utils.get_recap_data(video_id)

annotated_results, major_topics, minor_topics = await services.stream_plot.generate_data(stream_recap_data)

plot_object, annotated_results, plot_segments, category_locations = await services.stream_plot.process_data(stream_recap_data,  annotated_results, major_topics, minor_topics, video_id)

await services.stream_plot.generate_plot(plot_object)


# Prod Testing

In [ ]:
class StreamPlotController:
    async def generate_data(stream_recap_data: StreamRecapData):
        text_chunks_no_overlap = await services.stream_plot.data_gen.create_text_chunks(stream_recap_data.transcript, 0)

        text_chunk_batches = await services.stream_plot.data_gen.generate_text_chunk_batches(text_chunks_no_overlap)

        topic_annotations_str = await services.stream_plot.data_gen.annotate_major_minor_topics(stream_recap_data.recap)
        major_topics, minor_topics = services.stream_plot.data_gen.process_topic_annotations_str(topic_annotations_str)

        responses, annotated_results = await services.stream_plot.data_gen.annotate_all_batches(text_chunk_batches, topic_annotations_str)

        return annotated_results, major_topics, minor_topics
    
    async def process_data(stream_recap_data: StreamRecapData, annotated_results, major_topics, minor_topics, video_id):
        annotated_segments, category_locations = await services.stream_plot.data_processing.create_segments(stream_recap_data.linked_transcript, annotated_results, major_topics, stream_recap_data.transcript)

        plot_segments=await  services.stream_plot.data_processing.annotated_to_plot_segments(annotated_segments)

        plot_object=await  services.stream_plot.data_processing.create_plot_object(plot_segments, category_locations, video_id)

        return plot_object, annotated_segments, plot_segments, category_locations
    
    async def generate_plot(plot_object):
        return await services.stream_plot.data_plotting.generate_plot(plot_object)
    
    @classmethod
    async def run(self, video_id):
        stream_recap_data=await utils.get_recap_data(video_id)

        annotated_results, major_topics, minor_topics = await self.generate_data(stream_recap_data)

        plot_object, annotated_results, plot_segments, category_locations = await self.process_data(stream_recap_data,  annotated_results, major_topics, minor_topics, video_id)

        await self.generate_plot(plot_object)